## Start Node

In [1]:
import rclpy
from nimbro_utils.lazy import start_and_spin_node, stop_node

In [2]:
class MyNode(rclpy.node.Node):
    def __init__(self, context=None):
        super().__init__("test_depth_converter_node", context=context)

In [3]:
node, executor, context, thread = start_and_spin_node(MyNode, blocking=False)

> Starting node 'MyNode'


## Add DepthConverter

In [4]:
from nimbro_utils.lazy import DepthConverter

In [5]:
node.depth_converter = DepthConverter(node, settings={'depth_info_topic': "/gemini/depth/camera_info"})

#### The DepthConverter's settings can be obtained via `get_settings()` and updated via `set_settings()`.

In [6]:
node.depth_converter.get_settings()

{'depth_info_topic': '/gemini/depth/camera_info',
 'severity': 20,
 'suffix': 'depth_converter',
 'depth_info_update': None,
 'depth_scale': 0.001}

In [7]:
node.depth_converter.set_settings(settings={'severity': 20})

2025-09-10 18:41:18.529 [INFO] [test_depth_converter_node.depth_converter]: Initialized camera parameters


#### Utilities are provided to check if camera paraeters have been received, to wait for them, and to obtain them.

In [8]:
node.depth_converter.is_camera_received()

True

In [9]:
node.depth_converter.wait_for_camera()

In [10]:
node.depth_converter.get_camera()

{'frame': 'gemini_color_optical_frame',
 'model': 'rational_polynomial',
 'width': 1920,
 'height': 1080,
 'focal_x': 1037.986572265625,
 'focal_y': 1038.075439453125,
 'center_x': 965.96484375,
 'center_y': 533.9769897460938}

In [11]:
pixel_coordinates = node.depth_converter.get_pixels()
print(pixel_coordinates.shape)

(2073600, 2)


#### We use a SensorInterface to obtain a depth image from an Orbbec Gemini 335.

In [12]:
from nimbro_utils.lazy import SensorInterface
settings = {
    'names': ["depth"],
    'topics': ["/gemini/depth/image_raw/compressedDepth"],
    'types': ["CompressedImage"],
    'formats': ["passthrough"],
}
node.sensor_interface = SensorInterface(node, settings=settings)

2025-09-10 18:41:18.620 [INFO] [test_depth_converter_node.sensors]: Subscribing and caching topics: [('/gemini/depth/image_raw/compressedDepth', 'CompressedImage', 10)] (1)


In [13]:
success, message, depth_image = node.sensor_interface.get_data(source="depth")

In [14]:
print(success)
print(message)
print(type(depth_image))

True
Retrieved sensor data from source 'depth' with age '-0.018s' after waiting '0.274s'.
<class 'sensor_msgs.msg._compressed_image.CompressedImage'>


## Convert depth image to pointcloud

In [15]:
pointcloud = node.depth_converter.get_point_cloud(pixels=None, depth_msg=depth_image, filter_invalid=True)

In [16]:
pointcloud.shape

(1649323, 3)

#### Passing a depth image to `get_point_cloud()` causes it to be pre-processed (conversion, scaling, validation, etc.). When obtaining a different pointcloud from the same depth image (w.r.t. `pixels`) it must not be passed again.

In [17]:
node.depth_converter.is_depth_set()

True

In [18]:
pointcloud = node.depth_converter.get_point_cloud(pixels=[[0,0], [1,1]], depth_msg=None, filter_invalid=True)

In [19]:
pointcloud

array([[-2.71832352, -1.50253703,  2.921     ],
       [-2.71550942, -1.49972317,  2.921     ]])

#### Pre-processing can also be triggered explicitly.

In [20]:
node.depth_converter.set_depth(depth_msg=depth_image)

## Stop Node

In [21]:
stop_node(node, executor, context, thread)

> Stopped node 'MyNode'
